In [1]:
# import libraries to use
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def thresh_image(img, thresh=(0, 255)):
    filtered = img.copy()
    filtered[filtered <= thresh[0]] = 0
    filtered[filtered > thresh[1]] = 0
    return filtered

In [3]:
def to_binary(img):
    binary = np.zeros_like(img)
    binary[img > 0] = 1
    return binary

In [4]:
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate the x and y gradients
    grad_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    grad_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction,
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(grad_y), np.absolute(grad_x))
    filtered = absgraddir.copy()
    filtered[absgraddir <= thresh[0]] = 0
    filtered[absgraddir > thresh[1]] = 0
    return filtered

In [5]:
def mag_thresh(img, sobel_kernel=3, thresh=(0, 255)):
    grad_x = cv2.Sobel(img, cv2.CV_64F, 1, 0)
    grad_y = cv2.Sobel(img, cv2.CV_64F, 0, 1)
    # Calculate the magnitude 
    abs_grad = np.sqrt(grad_x**2 + grad_y**2)
    # Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_grad = np.uint8(255*abs_grad/np.max(abs_grad))
    # Filter pixel that do not fit within thresholds
    filtered = scaled_grad.copy()
    filtered[scaled_grad <= thresh[0]] = 0
    filtered[scaled_grad > thresh[1]] = 0
    return filtered

In [6]:
def thresholding(img, thresholds):
    # we asume image is read in BGR form
    rgb_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # convert from BGR to HSL
    hls_image = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    # Thresholding channel s from a hls image
    filt_s = thresh_image(hls_image[:,:,2], thresh=thresholds['thresh_s'])
    # Thresholding channel r from a rgb image
    filt_r = thresh_image(rgb_image[:,:,0], thresh=thresholds['thresh_r'])
    # Let's add values from both images filtered
    add_im = filt_r.astype(int)
    add_im += filt_s
    # Normalize results to 255
    add_im = add_im / add_im.max() * 255
    # Calculate gradient thresholded images
    gray = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)
    grad_mag = mag_thresh(gray, sobel_kernel=12, thresh=thresholds['thresh_mag'])
    grad_dir = dir_threshold(gray, sobel_kernel=15, thresh=thresholds['thresh_dir'])
    # Combine all images
    final_im = 1.8 * to_binary(thresh_image(add_im, thresh=thresholds['thresh_color'])) * 255
    final_im += 1 * (grad_dir / grad_dir.max() * 255)
    final_im += 2 * (grad_mag / grad_mag.max() * 255)
    # Normalize results to 255
    final_im = final_im / final_im.max() * 255
    # Create a thresholded binary image of the combined image.
    # we also applied a gaussian blur to smooth out noise
    blur_gray = cv2.GaussianBlur(final_im,(3, 3), 0)
    binary = to_binary(thresh_image(blur_gray, thresh=thresholds['thresh_final']))
    return binary

In [7]:
def line_mask(h=720, w=1280, point1=(0,0), point2=(720, 1280), band = 150):
    mask = np.zeros((h, w), np.int8)
    cv2.line(mask, point1, point2, (1,0,0), band)
    return mask

In [105]:
def find_lane(img, bandwidth, line):
    try:
        # Let's see where the lane starts at the bottom of the picture
        histogram = np.sum(img[int(img.shape[0]/3):,:], axis=0)
        # finding the right and left peak of the histogram would give a good approximation of where the lane is
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        # Create a mask for our region of interest
        left_mask= line_mask(h=img.shape[0], w=img.shape[1],
                             point1=(leftx_base, img.shape[0]),
                             point2=(leftx_base, 0),
                             band = bandwidth)
        right_mask = line_mask(h=img.shape[0], w=img.shape[1],
                               point1=(rightx_base, img.shape[0]), 
                               point2=(rightx_base, 0),
                               band = bandwidth)
        left_lane = cv2.bitwise_and(img, img, mask=left_mask)
        right_lane = cv2.bitwise_and(img, img, mask=right_mask)
        # fit a straight line through our points
        # Carefull with what is x and y depending if we use image or matrix form
        y_fit, x_fit = np.nonzero(left_lane) # y_fit is first component, rows in matrix
        l_lane_line = np.polyfit(y_fit, x_fit, 1) # x and y are inverted from a "normal" polynomial
        y_fit, x_fit = np.nonzero(right_lane) # y_fit is first component, rows in matrix
        r_lane_line = np.polyfit(y_fit, x_fit, 1) # x and y are inverted from a "normal" polynomial
        # Create reference point for new line
        left_base = (int(l_lane_line[0] * 720 + l_lane_line[1]) , 720)
        left_top = (int(l_lane_line[1]) , 0)
        right_base = (int(r_lane_line[0] * 720 + r_lane_line[1]) , 720)
        right_top = (int(r_lane_line[1]) , 0)
        # Repeat the masking to improve the result but now with the line obtained before
        # We'll also fit a degree two polynomial
        # Update the mask for our region of interest
        left_mask= line_mask(h=img.shape[0], w=img.shape[1], point1=left_base, point2=left_top)
        right_mask = line_mask(h=img.shape[0], w=img.shape[1], point1=right_base, point2=right_top)
        left_lane = cv2.bitwise_and(img, img, mask=left_mask)
        right_lane = cv2.bitwise_and(img, img, mask=right_mask)
        # fit a degree two polynomial through our points and points from previous n frames
        # Carefull with what is x and y depending if we use image or matrix form
        line.left_y_fit.appendleft(np.nonzero(left_lane)[0])
        line.left_x_fit.appendleft(np.nonzero(left_lane)[1])
        line.right_y_fit.appendleft(np.nonzero(right_lane)[0])
        line.right_x_fit.appendleft(np.nonzero(right_lane)[1])
        if len(line.right_x_fit) > line.n:
            line.left_y_fit.pop()
            line.left_x_fit.pop()
            line.right_y_fit.pop()
            line.right_x_fit.pop()
        y_fit = np.concatenate(line.left_y_fit)
        x_fit = np.concatenate(line.left_x_fit)
        l_polynomial = np.polyfit(y_fit, x_fit, 2) # x and y are inverted from a "normal" polynomial
        y_fit = np.concatenate(line.right_y_fit)
        x_fit = np.concatenate(line.right_x_fit)
        r_polynomial = np.polyfit(y_fit, x_fit, 2) # x and y are inverted from a "normal" polynomial
        # return polynomial coefficients
        return (l_polynomial, r_polynomial)
    except:
        return False

In [106]:
def plot_lane(img, lane):
    y_value = img.shape[0]
    ploty = np.linspace(0, y_value-1, num=y_value)
    l_polynomial, r_polynomial = lane
    left_fit = l_polynomial[0] * ploty ** 2 + l_polynomial[1] * ploty + l_polynomial[2]
    right_fit = r_polynomial[0] * ploty ** 2 + r_polynomial[1] * ploty + r_polynomial[2]
    # copy img to keep original
    lane_warp = img.copy()
    # zero the image
    lane_warp[:] = 0
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fit, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fit, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    cv2.fillPoly(lane_warp, np.int_([pts]), (0,255, 0))
    # unwarp the image to the original space
    lane = cv2.warpPerspective(lane_warp, Minv, (img.shape[1], img.shape[0]))
    result = cv2.addWeighted(img, 1, lane, 0.3, 0)
    return result

In [165]:
from collections import deque
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # number of frames to average
        self.n = 8
        # x values of left line to fit from n last frames
        self.left_x_fit = deque()
        # y values of left line to fit from n last frames
        self.left_y_fit = deque()
        # x values of right line to fit from n last frames
        self.right_x_fit = deque()
        # y values of right line to fit from n last frames
        self.right_y_fit = deque()
        #polynomial coefficients to use in current frame
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #alpha update strength
        self.alpha = 0.8
        #pixel scale
        lane_pixel_width = 960 - 320
        self.ym_per_pixel = 30 / 720
        self.xm_per_pixel = 3.7 / lane_pixel_width
    
    def update_line(self, lane):
        if self.current_fit[-1].any() and lane:
            self.current_fit = self.current_fit * (1 - self.alpha) +  np.array(lane) * self.alpha
            self.detected = True
        elif lane:
            self.current_fit = np.array(lane)
            self.detected = True
        else:
            self.detected = False
    
    def calc_radius_pos(self):
        # Calculate radius for both left and right
        # Scale pixels to meter and fit polynomial again
        y_value = 719 * self.ym_per_pixel # hardcoded for 720p video
        y_fit = np.concatenate(line.left_y_fit) 
        x_fit = np.concatenate(line.left_x_fit)
        l_polynomial_m = np.polyfit(y_fit * self.ym_per_pixel, x_fit * self.xm_per_pixel, 2)
        y_fit = np.concatenate(line.right_y_fit)
        x_fit = np.concatenate(line.right_x_fit)
        r_polynomial_m = np.polyfit(y_fit * self.ym_per_pixel, x_fit * self.xm_per_pixel, 2)
        # assigment for clarity
        A, B, C = l_polynomial_m
        R_left = (1 + (2 * A * y_value + B) ** 2) ** (3 / 2) / np.absolute(2 * A)
        A, B, C = r_polynomial_m
        R_right = (1 + (2 * A * y_value + B) ** 2) ** (3 / 2) / np.absolute(2 * A)
        # maybe an average is a better estimate
        self.radius_of_curvature = (R_right + R_left) / 2
        # calculate position here too, reuse of fit
        x_max = np.dot(r_polynomial_m, np.array([y_value**2, y_value, 1]))
        x_min = np.dot(l_polynomial_m, np.array([y_value**2, y_value, 1]))
        mid_lane = (x_max + x_min) / 2
        mid_image = (1280 * self.xm_per_pixel) / 2 # image with hardcoded
        self.line_base_pos = mid_image  - mid_lane
        return self.radius_of_curvature, self.line_base_pos


In [167]:
def process_frame(img, mtx, dist_coef, thresholds, line):
    un_dis = cv2.undistort(img, mtx, dist_coef, None, mtx)
    thresholded = thresholding(un_dis, thresholds)
    warped = cv2.warpPerspective(thresholded, M, (thresholded.shape[1], thresholded.shape[0]), flags=cv2.INTER_LINEAR)
    lane = find_lane(warped, 250, line)
    line.update_line(lane)
    im = plot_lane(img, line.current_fit)
    radius, pos = line.calc_radius_pos()
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(im,'Road Curvature {:.1f}m - Car Position {:.2f}m'.format(radius, pos), (200,50), font, 1,(255,255,255),2)
    return im

In [155]:
# Reading calibration from files
mtx = np.load('camera_matrix.npy')
dist_coef = np.load('distortion_coeff.npy')
# read perspective from files
M = np.load('direct_persp_trans.npy')
Minv = np.load('inverse_persp_trans.npy')

thresh_values = {'thresh_s':(100, 255), 'thresh_r':(190,255), 'thresh_color':(110,255),
                  'thresh_dir':(0.8,1.2), 'thresh_mag':(30,150), 'thresh_final':(110, 255)}

In [156]:
from moviepy.editor import VideoFileClip, ImageSequenceClip
from IPython.display import HTML
def process_video(img):
    # my code assumes image is read in BGR as it is by opencv. Image from moviepy come as RBG
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    image_bgr = process_frame(img, mtx, dist_coef, thresh_values, line)
    return cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

In [169]:
line = Line()
video_output = './project_video_processed.mp4'
clip = VideoFileClip('./project_video.mp4')
clip_processed = clip.fl_image(process_video) #NOTE: this function expects color images!!
%time clip_processed.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video ./project_video_processed.mp4
[MoviePy] Writing video ./project_video_processed.mp4


100%|█████████▉| 1260/1261 [09:12<00:00,  2.34it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./project_video_processed.mp4 

CPU times: user 1h 39min 38s, sys: 3min 19s, total: 1h 42min 58s
Wall time: 9min 12s


In [170]:
line = Line()
video_output = './challenge_video_processed.mp4'
clip = VideoFileClip('./challenge_video.mp4')
clip_processed = clip.fl_image(process_video) #NOTE: this function expects color images!!
%time clip_processed.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video ./challenge_video_processed.mp4
[MoviePy] Writing video ./challenge_video_processed.mp4


100%|██████████| 485/485 [03:07<00:00,  2.69it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./challenge_video_processed.mp4 

CPU times: user 33min 19s, sys: 1min 7s, total: 34min 26s
Wall time: 3min 7s


In [171]:
line = Line()
video_output = './harder_challenge_video_processed.mp4'
clip = VideoFileClip('./harder_challenge_video.mp4')
clip_processed = clip.fl_image(process_video) #NOTE: this function expects color images!!
%time clip_processed.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video ./harder_challenge_video_processed.mp4
[MoviePy] Writing video ./harder_challenge_video_processed.mp4


100%|█████████▉| 1199/1200 [09:04<00:00,  2.34it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./harder_challenge_video_processed.mp4 

CPU times: user 1h 33min 28s, sys: 3min 17s, total: 1h 36min 45s
Wall time: 9min 5s
